# Q* Learning with OpenAI Taxi
The goal of this game is to pick up the passenger at one location and drop him off to the goal as fast as possible.

There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another.

- You receive +20 points for a successful dropoff
- Lose 1 point for every timestep it takes.
- There is also a 10 point penalty for illegal pick-up and drop-off actions (if you don't drop the passenger in one of the 3 other locations)

In [1]:
# Bit of formatting because inline code is not styled very good by default:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

In [2]:
# Get necessary libraries
import numpy as np
import gym
import random

## Step 1: Create Environment from OpenAI Gym

In [3]:
env = gym.make("Taxi-v3")
env.reset()                    
env.render()

print("Action space: ", env.action_space)
print("Observation space: ", env.observation_space)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space:  Discrete(6)
Observation space:  Discrete(500)


## Step 2: Initialize Q table

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n

qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## Step 3: Set Hyperparameters

In [5]:
total_episodes = 50000        # Total episodes
learning_rate = 0.5           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.5                  # Discounting rate

epsilon = 1.0               # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01            # Exponential decay rate for exploration prob

## Step 4: Implement Q Learning Algorithm
Initialize Q-values arbitrarily for all state-action pairs <br>
Repeat for each episode:
- Choose an action in the current world state based on current Q-value estimates
- Take action and observe the outcome state and reward
- Update Q function based on the Bellman Equation

In [6]:
rewards = []

for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 6.11496
[[  0.           0.           0.           0.           0.
    0.        ]
 [ -1.98921992  -1.97838731  -1.98183416  -1.97846477  -1.95703125
  -10.97377904]
 [ -1.82849856  -1.67066765  -1.82815331  -1.65634677  -1.3125
  -10.63305664]
 ...
 [ -1.73303223  -1.65624805  -1.68359375  -1.68838501  -7.625
   -5.        ]
 [ -1.91552973  -1.909444    -1.92960811  -1.91103196  -7.7890625
   -7.765625  ]
 [ -0.875       -0.75        -0.75         8.99997187  -7.5
   -5.        ]]


## Step 5: Play OpenAI Taxi

In [7]:
env.reset()
rewards = []
total_test_episodes = 5

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        env.render()
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            #env.render()
            rewards.append(total_rewards)
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+-------